In [1]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lihem (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'reg_alpha': {
        'values': [0] + np.logspace(-5, -2, num=7).tolist()
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.00025118864
        },
    'batch_size': {
          'value': 64
        },
    'kernel_size': {
        'value': (3, 3)
        },
    'dropout': {
          'value': True
        },
    'pooling': {
          'value': 'max'
        },
    'batchnorm': {
          'value': True
        },
    'a_layers': {
          'value': 16
        },
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'a_layers': {'value': 16},
                'batch_size': {'value': 64},
                'batchnorm': {'value': True},
                'dropout': {'value': True},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'kernel_size': {'value': (3, 3)},
                'learning_rate': {'value': 0.00025118864},
                'pooling': {'value': 'max'},
                'reg_alpha': {'values': [0,
                                         1e-05,
                                         3.1622776601683795e-05,
                                         0.0001,
                                         0.00031622776601683794,
                                         0.001,
                                         0.0031622776601683794,
                                         0.01]}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: nj765c0f
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/nj765c0f


In [7]:
import tensorflow as tf

def create_model(kernel_size, dropout, pooling, batchnorm, n_layers, reg_alpha):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    if n_layers == 19:
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        if batchnorm:
            model.add(tf.keras.layers.BatchNormalization())


    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model


In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model(config["kernel_size"], config["dropout"], config["pooling"], config["batchnorm"], config["a_layers"], config["reg_alpha"])
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [9]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: miggc7tq with config:
wandb: 	a_layers: 16
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: max
wandb: 	reg_alpha: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 27s 34ms/step - loss: 1.5844 - accuracy: 0.4291 - top@3_accuracy: 0.7637 - val_loss: 1.4624 - val_accuracy: 0.4896 - val_top@3_accuracy: 0.8248
Epoch 2/100
704/704 [==============================] - 23s 33ms/step - loss: 1.0759 - accuracy: 0.6273 - top@3_accuracy: 0.8837 - val_loss: 0.9832 - val_accuracy: 0.6568 - val_top@3_accuracy: 0.9074
Epoch 3/100
704/704 [==============================] - 22s 32ms/step - loss: 0.8354 - accuracy: 0.7210 - top@3_accuracy: 0.9233 - val_loss: 0.9616 - val_accuracy: 0.6794 - val_top@3_accuracy: 0.8876
Epoch 4/100
704/704 [==============================] - 22s 31ms/step - loss: 0.6712 - accuracy: 0.7781 - top@3_accuracy: 0.9449 - val_loss: 0.6988 - val_accuracy: 0.7608 - val_top@3_accuracy: 0.9404
Epoch 5/100
704/704 [==============================] - 22s 31ms/step - loss: 0.5319 - accuracy: 0.8264 - top@3_accuracy: 0.9609 - val_loss: 0.7996 - val_accuracy: 0.7396 - val_top@3_accuracy: 0.9250
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▇▇▇▇█▇████▇███
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▂▁▂▁
epoch/top@3_accuracy,▁▅▆▆▇▇▇█████████████
epoch/val_accuracy,▁▄▅▇▆▇▇▇▇██▄███▇██▆█
epoch/val_loss,█▄▄▁▂▁▂▂▂▁▂▄▁▂▂▃▁▃▂▂
epoch/val_top@3_accuracy,▁▅▄▇▆▇▇▇▇██▄███▇▇█▇█
test_acc,▁


wandb: Agent Starting Run: em6t41zg with config:
wandb: 	a_layers: 16
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: max
wandb: 	reg_alpha: 1e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 25s 34ms/step - loss: 1.6804 - accuracy: 0.4277 - top@3_accuracy: 0.7630 - val_loss: 1.3944 - val_accuracy: 0.5268 - val_top@3_accuracy: 0.8512
Epoch 2/100
704/704 [==============================] - 25s 35ms/step - loss: 1.1394 - accuracy: 0.6344 - top@3_accuracy: 0.8897 - val_loss: 1.1254 - val_accuracy: 0.6476 - val_top@3_accuracy: 0.9110
Epoch 3/100
704/704 [==============================] - 24s 34ms/step - loss: 0.9006 - accuracy: 0.7256 - top@3_accuracy: 0.9265 - val_loss: 1.0966 - val_accuracy: 0.6804 - val_top@3_accuracy: 0.8798
Epoch 4/100
704/704 [==============================] - 24s 34ms/step - loss: 0.7387 - accuracy: 0.7839 - top@3_accuracy: 0.9483 - val_loss: 0.8167 - val_accuracy: 0.7482 - val_top@3_accuracy: 0.9370
Epoch 5/100
704/704 [==============================] - 24s 34ms/step - loss: 0.6153 - accuracy: 0.8276 - top@3_accuracy: 0.9638 - val_loss: 0.7837 - val_accuracy: 0.7728 - val_top@3_accuracy: 0.9376
Epoch

...

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▆▇▇▇▇▇▇▇▇███████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇██████████████████
epoch/val_accuracy,▁▂▅▆▆▆▇▆▆▆▇▇▇▇▇▇▇▆▇█▇█▇▇▇█████
epoch/val_loss,█▄▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁
epoch/val_top@3_accuracy,▁▃▅▇▇▆▇▆▆▆█▇█▇█▇▇▆▇█▆█▇▇▇█████
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1nkpl2t6 with config:
wandb: 	a_layers: 16
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: max
wandb: 	reg_alpha: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 26s 35ms/step - loss: 20.0187 - accuracy: 0.4239 - top@3_accuracy: 0.7689 - val_loss: 4.1994 - val_accuracy: 0.4656 - val_top@3_accuracy: 0.7860
Epoch 2/100
704/704 [==============================] - 24s 35ms/step - loss: 2.6934 - accuracy: 0.5838 - top@3_accuracy: 0.8677 - val_loss: 2.2131 - val_accuracy: 0.5434 - val_top@3_accuracy: 0.8372
Epoch 3/100
704/704 [==============================] - 24s 35ms/step - loss: 1.8092 - accuracy: 0.6518 - top@3_accuracy: 0.8955 - val_loss: 1.9341 - val_accuracy: 0.5594 - val_top@3_accuracy: 0.8606
Epoch 4/100
704/704 [==============================] - 24s 35ms/step - loss: 1.5165 - accuracy: 0.6993 - top@3_accuracy: 0.9152 - val_loss: 2.3337 - val_accuracy: 0.4758 - val_top@3_accuracy: 0.7524
Epoch 5/100
704/704 [==============================] - 25s 35ms/step - loss: 1.3716 - accuracy: 0.7343 - top@3_accuracy: 0.9252 - val_loss: 1.7463 - val_accuracy: 0.6432 - val_top@3_accuracy: 0.8658
Epoc

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇████████████████████████
epoch/val_accuracy,▁▃▃▁▄▅▅▆▇▇█▆▆▇█▇▇█▇███▇▇█▇▆▇█████▇▇█▇▇▇
epoch/val_loss,█▄▃▄▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▂▁▁
epoch/val_top@3_accuracy,▂▄▅▁▅▆▆▇▇█▇▇▆▇█▇▇▇▇▇██▇██▇▆▇███▇█▇▇▇▇▇▇
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
